# Requirements

- Putting src data from BQ, into the workspace bucket
- id_rsa key and placement in the ~ directory in Terra
- Study specific dirs created in the workspace bucket. 

# Data requirements
 - src datafiles are in the study specific workspace bucket dir
   - if outside of the workspace bring them in. Functions exist to create the sql needed to get the data into the workspace, to be run in the projects BigQuery Console.
 - {study_id}_validation.yaml exists 

# Process
- Don't run all the cells at once without reviewing them, parameters toward the bottom, will need to be updated first.


### TODOs
- Make this a terra workflow(s)

In [ ]:
pipeline = 'anvil_dbt_project'
repo='git@github.com:NIH-NCPI/anvil_dbt_project.git' #The ssh version
study_id = ''
org_id = 'anvil'
dbt_repo = 'anvil_dbt_project'
tgt_model_id = 'tgt_consensus_a'

ftd_schema = f'main_{study_id}_data'
tgt_schema = f'main_{study_id}_tgt_data'

gh_email = ''
gh_user = ''

In [ ]:
# Imports
from pathlib import Path
import os
import pandas as pd
import duckdb
from jinja2 import Template
import sys

bucket = os.environ['WORKSPACE_BUCKET']
con = duckdb.connect("/tmp/dbt.duckdb")

project_root = Path().resolve().parent
sys.path.append(str(project_root))
from scripts.general import *
paths = get_all_paths(study_id, dbt_repo, org_id, tgt_model_id, src_data_path=None)


validation_config = read_file(paths["validation_yml_path"])
study_config = read_file(paths["study_yml_path"])


In [ ]:
src_table_list = list(study_config["data_dictionary"].keys())

src_files_list = []
datasets = validation_config["datasets"].items()
dataset_names = list(validation_config["datasets"].keys())

    
for table in study_config["data_dictionary"].keys():
    for dataset, v in validation_config["datasets"].items():
        f_table = table
        if v['table_name_swap']:
            if f_table in v['table_name_swap'].keys():
                f_table = v['table_name_swap'].get(table)
        fn = v['filename']
        src_files_list.append(f"{f_table}_{fn}")
        
study_files = [f'{study_id}_study.yaml', f'{study_id}_validation.yaml']

for dd_table in study_config["data_dictionary"].values():
    study_files.append(dd_table['identifier'])
    if validation_config["bucket_seeds"]:
        for file in validation_config["bucket_data_files"]:
            study_files.append(file)

seeds_files = []
if validation_config["bucket_seeds"]:
    for file in validation_config["bucket_seeds"]:
        seeds_files.append(file)

# print("Source Tables:", src_table_list)
# print("Source Files List:", src_files_list)
# print("Study Files:", study_files)
# print("Seed Files:", seed_files)

In [ ]:
'''
If starting a new pipeline env
- After putting your Private id_rsa key file in the home dir in Terra
1. Set up GH and terminal configurations
- Run 'run_initial_setup' in this cell
- Go to terminal and run:
    - 'source ~/.bash_profile' - A list of available commands should show up
    - 'setup_ssh'
    - 'clone_repo'
    - 'setup_data'
- At this point you should be able to connect to GitHub and swap branches
'''
# run_initial_setup(paths, gh_user, gh_email, pipeline)




'''
Get the dds and config files from the bucket
'''
# get_study_files(study_files, seeds_files, paths) 




"""
Run the following if you only want to pull in the data files from the bucket. Run this 
if there are no 'partial' files to combine.
"""
# copy_data_from_bucket(paths['bucket_study_dir'], src_table_list, paths['src_data_dir'])




'''
Put study files into the bucket.
'''
store_study_files(study_files, seeds_files, paths)




'''
Export tgt data to csvs in the output dir. Then send the files to the harmonized dir in the bucket
'''
# copy_to_csv_and_export_to_bucket(tgt_schema, paths)




"""
Convert files in data dir into utf-8. Add to the appropriate list, to save the changes in the bucket.
"""
# input_filepath = f'{seeds_dir}/value_sets/RoleCodeValueSet.csv'
# output_filepath = f'{seeds_dir}/value_sets/RoleCodeValueSet.csv'
# delimiter = '\t'
# encoding = 'latin1'
# convert_csv_to_utf8(input_filepath, output_filepath, delimiter, encoding)

print('Completed')